In [48]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split

In [49]:
import keras
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Activation
from keras.models import Sequential
from keras.utils import to_categorical

In [55]:
data = pd.read_csv('./df_final.csv')

#data = data.drop(['ID','Total', 'HP', 'Attack', 'Defense','Sp. Atk','Sp. Def',
#                  'Speed', 'Generation', 'Legendary'], axis='columns')

In [ ]:
data.to_csv('df_final.csv')

In [ ]:
len(manypoke)

In [ ]:
bug['Type1'] = "Bug"
grass['Type1'] = "Grass"
fire['Type1'] = "Fire"
water['Type1'] = "Water"
normal['Type1'] = "Normal"
poison['Type1'] = "Poison"
ele['Type1'] = "Electric"
ground['Type1'] = "Ground"
fairy['Type1'] = "Fairy"
fight['Type1'] = "Fight"
psy['Type1'] = "Psychic"
rock['Type1'] = "Rock"
ghost['Type1'] = "Ghost"
ice['Type1'] = "Ice"
drag['Type1'] = "Dragon"
dark['Type1'] = "Dark"
steel['Type1'] = "Steel"
fly['Type1'] = "Flying"

In [ ]:
data.append([bug, grass, fire, water, normal, poison, ele, ground, fairy, fight, psy, rock, ghost, ice, drag, dark, steel, fly], sort=True)

----

In [56]:
mask = np.random.rand(len(data)) < 0.8

In [57]:
train = data[mask].reset_index(drop=True)
test = data[~mask].reset_index(drop=True)

In [ ]:
validmask = np.random.rand(len(train)) < 0.8
valid = train[~validmask].reset_index(drop=True)
train = train[validmask].reset_index(drop=True)

In [46]:
train.index

RangeIndex(start=0, stop=7994, step=1)

In [47]:
test.index

RangeIndex(start=0, stop=2006, step=1)

In [ ]:
len(train)

In [ ]:
len(test)

In [98]:
gen = ImageDataGenerator(vertical_flip=True, validation_split=0.2)

In [99]:
traingen = gen.flow_from_dataframe(train, 
                                   directory='./images/comp/', 
                                   x_col='fileid', 
                                   y_col='Type1',
                                   class_mode='categorical',
                                   target_size=(224,224),
                                   subset='training')
validgen = gen.flow_from_dataframe(train, 
                                   directory='./images/comp/', 
                                   x_col='fileid', 
                                   y_col='Type1',
                                   class_mode='categorical',
                                   target_size=(224,224),
                                   subset='validation')

Found 16094 images belonging to 18 classes.
Found 4217 images belonging to 18 classes.


In [100]:
testdatagen = ImageDataGenerator(vertical_flip=False)
testgen = testdatagen.flow_from_dataframe(test, directory='./images/comp/', x_col='fileid', y_col='Type1', target_size=(224,224))

Found 5182 images belonging to 18 classes.


-----

In [74]:
from keras.applications.densenet import DenseNet121

In [75]:
pretrainmod = DenseNet121(weights='imagenet')

In [101]:
model = Sequential()
model.add(Conv2D(32, kernel_size=2, padding='same', activation='relu', 
                 input_shape=(224, 224, 3)))
model.add(Conv2D(32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=2, activation='relu'))
model.add(Conv2D(64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(18, activation='softmax'))

In [102]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [103]:
STEP_SIZE_TRAIN=traingen.n//traingen.batch_size
STEP_SIZE_VALID=validgen.n//validgen.batch_size
step_size_test=testgen.n//testgen.batch_size

In [104]:
model.fit_generator(generator=traingen,
                    epochs=10,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                   validation_data=validgen,
                   validation_steps=STEP_SIZE_VALID)

Epoch 1/10
502/502 [==============================] - 2213s 4s/step - loss: 13.7982 - val_loss: 14.0841
Epoch 2/10
502/502 [==============================] - 2217s 4s/step - loss: 13.8102 - val_loss: 14.0807
Epoch 3/10
502/502 [==============================] - 2081s 4s/step - loss: 13.8110 - val_loss: 14.0846
Epoch 4/10
370/502 [=====================>........] - ETA: 9:46 - loss: 13.8022

KeyboardInterrupt: 

In [96]:
pretrainmod.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
pretrainmod.fit_generator(generator=traingen,
                    epochs=10,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                   validation_data=validgen,
                   validation_steps=STEP_SIZE_VALID)

In [ ]:
testgen.reset()
pred = model.predict_generator(testgen, steps = step_size_test, verbose=1)

In [ ]:
pred[2]

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
predicted_class_indices

In [ ]:
labels = (testgen.class_indices)
print(labels)

In [ ]:
labels = dict((v,k) for k,v in labels.items())
print(labels)

In [ ]:
predictions = [labels[k] for k in predicted_class_indices]
print(predictions[-1])

In [ ]:
len(predictions)

In [ ]:
len(testgen.filenames)

In [ ]:
filenames=testgen.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

In [ ]:
results.groupby

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
bug = os.listdir('images/scraped/bug/resize/')
grass = os.listdir('images/scraped/grass/resize/')
fire = os.listdir('images/scraped/fire/resize/')
water = os.listdir('images/scraped/water/resize/')
normal = os.listdir('images/scraped/normal/resize/')
poison = os.listdir('images/scraped/poison/resize/')
ele = os.listdir('images/scraped/ele/resize/')
ground = os.listdir('images/scraped/ground/resize/')
fairy = os.listdir('images/scraped/fairy/resize/')
fight = os.listdir('images/scraped/fight/resize/')
psy = os.listdir('images/scraped/psy/resize/')
rock = os.listdir('images/scraped/rock/resize/')
ghost = os.listdir('images/scraped/ghost/resize/')
ice = os.listdir('images/scraped/ice/resize/')
drag = os.listdir('images/scraped/dragon/resize/')
dark = os.listdir('images/scraped/dark/resize/')
steel = os.listdir('images/scraped/steel/resize/')
fly = os.listdir('images/scraped/fly/resize/')

In [ ]:
bug = pd.DataFrame(bug, columns=['fileid'])
grass = pd.DataFrame(grass, columns=['fileid'])
fire = pd.DataFrame(fire, columns=['fileid'])
water = pd.DataFrame(water, columns=['fileid'])
normal = pd.DataFrame(normal, columns=['fileid'])
poison = pd.DataFrame(poison, columns=['fileid'])
ele = pd.DataFrame(ele, columns=['fileid'])
ground = pd.DataFrame(ground, columns=['fileid'])
fairy = pd.DataFrame(fairy, columns=['fileid'])
fight = pd.DataFrame(fight, columns=['fileid'])
psy = pd.DataFrame(psy, columns=['fileid'])
rock = pd.DataFrame(rock, columns=['fileid'])
ghost = pd.DataFrame(ghost, columns=['fileid'])
ice = pd.DataFrame(ice, columns=['fileid'])
drag = pd.DataFrame(drag, columns=['fileid'])
dark = pd.DataFrame(dark, columns=['fileid'])
steel = pd.DataFrame(steel, columns=['fileid'])
fly = pd.DataFrame(fly, columns=['fileid'])

In [ ]:
manypoke = os.listdir('images/comp/manypoke/resize')